"user will give you". Is it persona or still relavant answers or delimiters?

In [ ]:
from openai import OpenAI

client = OpenAI()

test = client.chat.completions.create(
model =  "gpt-3.5-turbo",
messages = [
            {"role": "system", "content": "نتا خبير فلهجة الدارجة المغربية وفالترجمة من الدارجة لنجليزية." + 
             ":عفاك تبع الشروط الماجية" +
             "ترجم الفعل المعطي بالدارجة المغربية لنجليزية كلمة بكلمة." +
             "مخصش يكون المخرج فعل مصدري; بلاما تعطي تفسيرات."
             },
            {"role": "user", "content": 'داق'},
],
temperature=0,
)

test2 = client.chat.completions.create(
model =  "gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are an expert in Moroccan Darija dialect and in translating Darija to English." +
            {"role": "user", "content": 'داق'},
        ],
temperature=0, 
)
test = test.choices[0].message.content
print(test)
test2 = test2.choices[0].message.content
print(test2)

"Purpose of the translation: To study the accuracy of LLMs on Low resource languages, particularly Darija." + 
"Target audience: Computational linguists" 

In [ ]:
from openai import OpenAI

client = OpenAI()

test = client.chat.completions.create(
model =  "gpt-3.5-turbo",
messages = [
            {"role": "system", "content": "ترجم الفعل المعطي بالدارجة المغربية لنجليزية كلمة بكلمة." + 
             "مخصش يكون المخرج فعل مصدري. بلاما تعطي تفسيرات."},
            {"role": "user", "content": 'داق'},
],
temperature=0, # deterministic
)

test2 = client.chat.completions.create(
model =  "gpt-3.5-turbo",
        messages=[
        {
            "role": "system", 
            "content": "Translate the provided Moroccan Darija language verbs into English using word-by-word translation." + 
            "Please fulfill the following conditions when translating."+
            "The translation must contain only the lemma of the translated verb. It should not contain 'to' for example"+
            "Purpose of the translation: To study the accuracy of LLMs on Low resource languages, particularly Darija." + 
            "Target audience: Computational linguists" 
        },
        {
            "role": "user", 
            "content":  f'{verb}'
        }
        ],
temperature=0, 
)
test = test.choices[0].message.content
print(test)
test2 = test2.choices[0].message.content
print(test2)

chack previous work with stratified 

should the transliteration be any more checked within itself between the two results?

there must be a clearance of all useless work.

work from now on is with average. Makes sense than threshold.